# Final Project Phase 2 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 2 of the Final Project. Answer all statements addressed below as specified in the instructions for the project, covering all necessary details. Please be clear and concise in your answers. Each response should be at most 3 sentences. Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

## Jupyter Notebook Quick Tips
Here are some quick formatting tips to get you started with Jupyter Notebooks. This is by no means exhaustive, and there are plenty of articles to highlight other things that can be done. We recommend using HTML syntax for Markdown but there is also Markdown syntax that is more streamlined and might be preferable. 
<a href = "https://towardsdatascience.com/markdown-cells-jupyter-notebook-d3bea8416671">Here's an article</a> that goes into more detail. (Double-click on cell to see syntax)

# Heading 1
## Heading 2
### Heading 3
#### Heading 4
<br>
<b>BoldText</b> or <i>ItalicText</i>
<br> <br>
Math Formulas: $x^2 + y^2 = 1$
<br> <br>
Line Breaks are done using br enclosed in < >.
<br><br>
Hyperlinks are done with: <a> https://www.google.com </a> or 
<a href="http://www.google.com">Google</a><br>

# Data Collection and Cleaning
You are required to provide data collection and cleaning for the three (3) minimum datasets. Create a function for each of the following sections that reads or scrapes data from a file or website, manipulate and cleans the parsed data, and writes the cleaned data into a new file. 

Make sure your data cleaning and manipulation process is not too simple. Performing complex manipulation and using modules not taught in class shows effort, which will increase the chance of receiving full credit.


## Data Sources
Include sources (as links) to your datasets. Add any additional data sources if needed. Clearly indicate if a data source is different from one submitted in your Phase I, as we will check that it satisfies the requirements.
* Downloaded Dataset Source: https://fdc.nal.usda.gov/download-datasets (FNDDS JSON)

* Web Collection #1 Source: https://fdc.nal.usda.gov/download-datasets (Foundational foods JSON)

* Web Collection #2 Source: https://www.nutritionvalue.org/




## Downloaded Dataset Requirement

Fill in the predefined functions with your data scraping/parsing code. You may modify/rename each function as you seem fit, but you must provide at least 3 separate functions that clean each of your required datasets.


In [ ]:
Note: The following method relies on the out put of the clean lab data. I essentialy made a three step cleaning process where each method 
recieves and support another in a way.


In [1]:
import re
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import csv
import time
import random

def clean_survey(url, foundation_clean): # args  url is survey_parsed and the other is cleaned foundation(lab) data
    df = pd.read_csv(url, encoding="latin1")
    fdf = pd.read_csv(foundation_clean, encoding="latin1")

    chosen_nutrients = {  #fixing inconsistency with naming and setting it to a specific more commonly utilized name and also displaying the amount from abbreviation to full name
            "Iron, Fe": "Iron(Milligrams)",
            "Calcium, Ca": "Calcium(Milligrams)",
            "Potassium, K": "Potassium(Milligrams)",
            "Vitamin A, RAE": "Vitamin A(Micrograms)",
            "Vitamin C, total ascorbic acid": "Vitamin C(Milligrams)",
            "Vitamin D (D2 + D3)": "Vitamin D(Micrograms)",
            "Protein": "Protein(Grams)",
            "Zinc, Zn": "Zinc(Milligrams)",
            "Magnesium, Mg": "Magnesium(Milligrams)",
            "Vitamin B-12": "Vitamin B-12(Micrograms)"
        }

    df_filter = df[df["nutrient_name"].isin(chosen_nutrients.keys())]# filtering such that it is our nutreints from above table
    pivot_df = df_filter.pivot_table(index=["fdcId", "description"],columns="nutrient_name",values="amount")
    pivot_df = pivot_df.reset_index()  #fixes indexes
    pivot_df = pivot_df.rename(columns = chosen_nutrients) #fixes headers

    pivot_df = pivot_df.round(2) #can be considered an inconsistency fix and for simplicity
    word_filter = first_word_filter(pivot_df, fdf)

    word_filter.to_csv("cleaned_survey_nutrients.csv", index=False)
    return word_filter
    


def first_word_filter(survey_df, foundation_df):  ##potential inconsistency fix,since foundation foods have a limit of how much data/foods they have I adjust such that they are in survey.
    foundation_first = foundation_df["description"].str.split().str[0].str.lower().unique()
    survey_filtered = survey_df[survey_df["description"].str.split().str[0].str.lower().isin(foundation_first)]
    return survey_filtered

def clean_labData(json_input):  # please read above note in previous cell


#### for tmrw fix the grams or micrograms and make sure to stay consistent.

    chosen_nutrients = {  #fixing inconsistency with naming and setting it to a specific more commonly utilized name and also displaying the amount from abbreviation to full name
            "Iron, Fe": "Iron(Milligrams)",
            "Calcium, Ca": "Calcium(Milligrams)",
            "Potassium, K": "Potassium(Milligrams)",
            "Vitamin A, RAE": "Vitamin A(Micrograms)",
            "Vitamin C, total ascorbic acid": "Vitamin C(Milligrams)",
            "Vitamin D (D2 + D3)": "Vitamin D(Micrograms)",
            "Protein": "Protein(Grams)",
            "Zinc, Zn": "Zinc(Milligrams)",
            "Magnesium, Mg": "Magnesium(Milligrams)",

            "Vitamin B-12": "Vitamin B-12(Micrograms)"
        }
    with open(json_input, "r", encoding="latin1") as file:

        food_data = json.load(file)

    res = []
    for food in food_data["FoundationFoods"]:

        nutrients_dict = {"description": food["description"]}
        for label in chosen_nutrients.values(): #inconsistency fix, and also a place holder if the nutrient is not there
            nutrients_dict[label] = 0

        for nutrient in food["foodNutrients"]:
            nutrient_name = nutrient["nutrient"]["name"]
            if nutrient_name in chosen_nutrients:
                nutrients_dict[chosen_nutrients[nutrient_name]] = nutrient["amount"]

        res.append(nutrients_dict)

    df = pd.DataFrame(res)   #all keys becomes the first row(header) and values are listed in column
    df= df.round(2)
    df.to_csv("cleaned_foundation_nutrients.csv", index=False, encoding="latin1")
    print("worked central")
    return df







############ Function Call ############


clean_labData("FoodData_Central_foundation_food_json_2025-04-24.json")   #please read above note in previous cell
clean_survey("survey_parsed_nutrients.csv", "cleaned_foundation_nutrients.csv")

worked central


nutrient_name,fdcId,description,Calcium(Milligrams),Iron(Milligrams),Magnesium(Milligrams),Potassium(Milligrams),Protein(Grams),Vitamin A(Micrograms),Vitamin B-12(Micrograms),Vitamin C(Milligrams),Vitamin D(Micrograms),Zinc(Milligrams)
0,2705384,"Milk, NFS",125.0,0.00,12.0,156.0,3.33,58.0,0.56,0.1,1.1,0.43
1,2705385,"Milk, whole",123.0,0.00,12.0,150.0,3.27,32.0,0.54,0.0,1.1,0.42
2,2705386,"Milk, reduced fat (2%)",126.0,0.00,12.0,159.0,3.36,83.0,0.55,0.2,1.1,0.43
3,2705387,"Milk, low fat (1%)",126.0,0.00,12.0,159.0,3.38,58.0,0.61,0.0,1.1,0.43
4,2705388,"Milk, fat free (skim)",132.0,0.00,12.0,167.0,3.43,64.0,0.58,0.0,1.1,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...
5416,2710800,"Cabbage, cooked, as ingredient",44.0,0.07,15.0,216.0,1.00,5.0,0.00,35.7,0.0,0.22
5417,2710801,"Cauliflower, cooked, as ingredient",23.0,0.44,16.0,311.0,2.00,0.0,0.00,42.7,0.0,0.28
5418,2710802,"Eggplant, cooked, as ingredient",10.0,0.25,15.0,246.0,1.05,1.0,0.00,2.0,0.0,0.17
5419,2710803,"Green beans, cooked, as ingredient",42.0,0.68,29.0,302.0,2.05,35.0,0.00,10.8,0.0,0.36


In [ ]:
Note: since some of the lines were removed the left most column dosent indicate any sequence,
it was the place number from original file and also helps me track back to it if neccesary(hence kept_

## Web Collection Requirement \#1


In [2]:
def clean_labData(json_input):


#### for tmrw fix the grams or micrograms and make sure to stay consistent.

    chosen_nutrients = {  #fixing inconsistency with naming and setting it to a specific more commonly utilized name and also displaying the amount from abbreviation to full name
            "Iron, Fe": "Iron(Milligrams)",
            "Calcium, Ca": "Calcium(Milligrams)",
            "Potassium, K": "Potassium(Milligrams)",
            "Vitamin A, RAE": "Vitamin A(Micrograms)",
            "Vitamin C, total ascorbic acid": "Vitamin C(Milligrams)",
            "Vitamin D (D2 + D3)": "Vitamin D(Micrograms)",
            "Protein": "Protein(Grams)",
            "Zinc, Zn": "Zinc(Milligrams)",
            "Magnesium, Mg": "Magnesium(Milligrams)",

            "Vitamin B-12": "Vitamin B-12(Micrograms)"
        }
    with open(json_input, "r", encoding="latin1") as file:

        food_data = json.load(file)

    res = []
    for food in food_data["FoundationFoods"]:

        nutrients_dict = {"description": food["description"]}
        for label in chosen_nutrients.values(): #inconsistency fix, and also a place holder if the nutrient is not there
            nutrients_dict[label] = 0

        for nutrient in food["foodNutrients"]:
            nutrient_name = nutrient["nutrient"]["name"]
            if nutrient_name in chosen_nutrients:
                nutrients_dict[chosen_nutrients[nutrient_name]] = nutrient["amount"]

        res.append(nutrients_dict)

    df = pd.DataFrame(res)   #all keys becomes the first row(header) and values are listed in column
    df= df.round(2)
    df.to_csv("cleaned_foundation_nutrients.csv", index=False, encoding="latin1")
    print("worked central")
    return df



############ Function Call ############
clean_labData("FoodData_Central_foundation_food_json_2025-04-24.json")

worked central


,description,Iron(Milligrams),Calcium(Milligrams),Potassium(Milligrams),Vitamin A(Micrograms),Vitamin C(Milligrams),Vitamin D(Micrograms),Protein(Grams),Zinc(Milligrams),Magnesium(Milligrams),Vitamin B-12(Micrograms)
0,"Hummus, commercial",2.41,41.0,289.0,1.0,0.0,0.0,7.35,1.38,71.10,0.00
1,"Tomatoes, grape, raw",0.33,11.0,260.0,0.0,27.2,0.0,0.83,0.20,11.90,0.00
2,"Beans, snap, green, canned, regular pack, drai...",0.78,36.0,97.0,0.0,0.0,0.0,1.04,0.19,12.70,0.00
3,"Frankfurter, beef, unheated",1.14,15.0,343.0,3.0,0.0,0.0,11.70,2.06,11.50,0.97
4,"Nuts, almonds, dry roasted, with salt added",3.17,273.0,684.0,2.0,0.0,0.0,20.40,2.80,258.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
335,"Green onion, (scallion), bulb and greens, root...",1.04,59.4,232.0,0.0,0.0,0.0,0.67,0.25,18.90,0.00
336,"Shallots, bulb, peeled, root removed, raw",0.33,26.1,252.0,0.0,0.0,0.0,1.38,0.28,15.10,0.00
337,"Juice, prune, shelf-stable",0.44,16.2,216.0,0.0,0.0,0.0,0.42,0.12,15.20,0.00
338,"Juice, pomegranate, from concentrate, shelf-st...",0.03,11.0,166.0,0.0,0.0,0.0,0.00,0.08,6.86,0.00


## Web Collection Requirement \#2

In [ ]:
Note: The following method relies on the out put of the clean lab data. I essentialy made a three step cleaning process where each method 
recieves and support another in a way.

In [3]:
import re
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import csv
import time
import random



def scrape_cleaner(cleanedfoundation_csv, output_csv="calories.csv"):
    base_url = "https://www.nutritionvalue.org"
    headers = {"User-Agent": "Mozilla/5.0"}


    df = pd.read_csv(cleanedfoundation_csv, header=None, encoding = "latin1")
    first_words = df[0].str.split(',').str[0].str.lower().str.strip()
    unique_words = sorted(set(first_words)) #removed duplicate words so it dosent send extra requests, and also alphabetical
    
    result = []

    with open(output_csv, mode="w", newline="", encoding="latin1") as f:
        writer = csv.writer(f)
        writer.writerow(["First Word", "Average Calories per 100g"])

        for food in unique_words:
            time.sleep(random.uniform(4.2, 5.1)) # to avoid getting blocked
            search_url = f"https://www.nutritionvalue.org/search.php?food_query={food}"
            res = requests.get(search_url, headers=headers)
            soup = BeautifulSoup(res.text, "html.parser")

            result_links = soup.select("a.table_item_name")[:1]  # only first search result(I wouldve done more, but I keep getting flagged for so many requests)  * pulls a tags with class 'table_item_name'

            calories_list = []

            for link in result_links:
                food_url = base_url + link['href']
                food_res = requests.get(food_url, headers=headers)
                food_soup = BeautifulSoup(food_res.text, "html.parser")



                cal_spot = food_soup.select_one("td#calories.right")
                text = cal_spot.text.strip()

                calories = float(text)


                span = food_soup.select_one("span#serving-size")
                grams = None

                if span:
                    grams_text = span.text.strip().split()[0]
                    grams = float(grams_text) #idk why it runs error when i cast into int

                if grams:  #inconsistency fix, adjusting for standard nutriton portion
                    cal_100g = (calories / grams) * 100
                    calories_list.append(cal_100g)

            if calories_list:
                calories_100g = int(calories_list[0])
                writer.writerow([food, calories_100g])
                result.append([food, calories_100g])
                print(f"{food}: {calories_100g}")
            else:
                writer.writerow([food, "No data"])
                
                result.append([food, "No data"])
                print(f"{food}: No data")
        return pd.DataFrame(result, columns=["First Word", "Average Calories per 100g"])





############ Function Call ############
scrape_cleaner("cleaned_foundation_nutrients.csv")

almond butter: 612
almond milk: 43
apple juice: 48
apples: 52
applesauce: 75
apricot: 47
arugula: 25
asparagus: 20
avocado: 166
bananas: 88
beans: 31
beef: 198
beets: 42
bison: 145
blackberries: 43
blackeye pea: 285
blueberries: 56
bread: 272
broccoli: 34
brussels sprouts: 43
buckwheat: 342
bulgur: 82
butter: 720
buttermilk: 62
cabbage: 24
carrots: 40
cauliflower: 25
celery: 13
cheese: 403
cherries: 63
chia seeds: 492
chicken: 143
chickpeas: 386
chickpeas (garbanzo beans: 378
collards: 33
cookies: 465
corn: 86
corn flour: 360
cottage cheese: 87
cranberry juice: 45
cream: 340
cream cheese: 340
crustaceans: 84
cucumber: 15
description: 0
egg: 144
eggplant: 25
eggs: 144
einkorn: 354
farro: 369
figs: 73
fish: 911
flaxseed: 550
flour: 340
fonio: 377
frankfurter: 290
garlic: 149
grape juice: 64
grapefruit juice: 38
grapes: 26
green onion: 26
ham: 267
hummus: 240
juice: 22
kale: 35
ketchup: 117
khorasan: 337
kiwifruit: 61
kiwifruit (kiwi): 68
lamb: 285
leeks: 60
lentils: 352
lettuce: 15
manda

,First Word,Average Calories per 100g
0,almond butter,612
1,almond milk,43
2,apple juice,48
3,apples,52
4,applesauce,75
...,...,...
127,tomato juice,15
128,tomatoes,26
129,turkey,147
130,wild rice,356


## Additional Dataset Parsing/Cleaning Functions

Write any supplemental (optional) functions here.

In [ ]:
def extra_source1():
    pass

    
############ Function Call ############
extra_source1()

In [ ]:
# Define further extra source functions as necessary

#Inconsistencies
For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1.  fixing inconsistency with naming where certain datasets or webs use certain names such as some utilizing Vitamin C, other Ascorbic acid(they are both same thing), hence we chose the rather traditional approach of "Vitamin" or the direct mineral name. (applied to clean surveys method)

2. Same as 1, however applied to clean lab data: fixing inconsistency with naming where certain datasets or webs use certain names such as some utilizing Vitamin C, other Ascorbic acid(they are both same thing), hence we chose the rather traditional approach of "Vitamin" or the direct mineral name.

3. To handle missing nutrients efficiently in clea lab Data, a value of 0 is used as a placeholder instead of leaving empty entries.

4. Due to standardization of 100g throughout the survey and lab datasets, we adjusted for the various serving sizes when web scraping such that it meets 100g standards and calculation to maintain consistency. Applied to scrape cleaner.

5. (if applicable)
